# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [ ]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from data.plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf
rospy.init_node('fixed_tf_broadcaster')

In [ ]:
# how many sensors would you like to distribute?
sensorsToDistribute = 11

# which mesh would you like to use?
stl_file = 'roboy_models/TestCube/stls/monkey.stl'
#stl_file = 'roboy_models/TestCube/stls/IcoSphere_360.stl'

In [ ]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=0*np.pi)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[1,0,0], angle= -np.pi)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

In [ ]:
from data.rvizMeshVis import meshVisualization

scale = 0.01
position = [0,0,0]
global orientationMesh
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

In [ ]:
for i in range(5):
    meshVisualization(orientationMesh, stl_file, color=(1.,1.,1.,0.9))

# Preprocess data

In [ ]:
from data.trisByDistance import *

#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles
triangles = scale * np.matrix(mesh1.points)

global trianglesBackup
trianglesBackup = triangles

global sortedTriangles

lighthouses = [LH1, LH2]
sortedTriangles = []

normalsNotNorm = mesh1.normals
global normals
normals = []

for normal in normalsNotNorm:
    normals.append(1/np.linalg.norm(normal,2)*normal)
    
normals = np.matrix(normals)
normals = scale * normals

for l in lighthouses:
    tris = trisByMinDistanceSortedMap(triangles, l[0])
    sortedTriangles.append(tris)

#vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
print('')
#print('%d vertices' %len(vertices))
#print('')
print('%d sensors in centers of triangles' %len(sensors))
print('')
print('%d normals' %len(normals))
#print(normals)

In [ ]:
from data.rvizNormalsVis import NormalsVisual

#NormalsVisual(sensors,normals)

# GA

In [ ]:
from deap import algorithms, base, creator, tools

In [ ]:
#sensors to dict
global sensor_dict
sensor_dict =  list(zip(range(len(sensors)), sensors.tolist()))

global sensorDictBackup
sensorDictBackup = sensor_dict


In [ ]:
from data.rvizSensorVis import sensorVisualization

#color = (r,g,b,a)
sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1,)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()

In [ ]:
from data.randomSensor import randomSensor

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, sensorsToDistribute)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.attr_bool()

In [ ]:
toolbox.individual()

# Evaluation (Fitness) Function

In [ ]:
from data.RayIntersectsTriangle import rayIntersectsTriangle, rayIntersectsTriangleVisual

def FitnessFunction(sensors):
    
    br = tf.TransformBroadcaster()
    br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                     (quat1[0], quat1[1], quat1[2], quat1[3]),
                     rospy.Time.now(),
                     "lighthouse1",
                     "world")
    br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                     (quat2[0], quat2[1], quat2[2], quat2[3]),
                     rospy.Time.now(),
                     "lighthouse2",
                     "world")

    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0.0
    visibleLH2 = 0.0
    angleLH1 = []

    for i, nmbr_sensor in enumerate(sensors):
        sensor = sensor_dict[nmbr_sensor][1]

        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse1')
        distLH1 = np.linalg.norm(np.asarray(sensor) - LH1_array)
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse2')
        distLH2 = np.linalg.norm(np.asarray(sensor) - LH2_array)
        
        # get angle between lighthouse vector and normal
        normal = np.squeeze(np.asarray(normals[nmbr_sensor]))
        #LH1
        sensorToLH1 = sensor + (LH1_array - sensor)
        angleLH1 = np.dot(sensorToLH1,normal)/(np.linalg.norm(sensorToLH1)*np.linalg.norm(normal))
        #angleLH1 = np.arccos(angleLH1)
        #LH2
        sensorToLH2 = sensor + (LH2_array - sensor)
        angleLH2 = np.dot(sensorToLH2,normal)/(np.linalg.norm(sensorToLH2)*np.linalg.norm(normal))
        #angleLH2 = np.arccos(angleLH2)
        
        # Might be changed to something different
        # Calculate visibility factor depending on angle between normal and lighthouse
        visFactorLH1 = angleLH1#np.cos(angleLH1)
        visFactorLH2 = angleLH2#np.cos(angleLH2)
        
        #print("Sensor %d has VisFactor %f mit LH1"%(nmbr_sensor, visFactorLH1))
        #print("Sensor %d has VisFactor %f mit LH2"%(nmbr_sensor, visFactorLH2))
        
        
        #print('interDist');print(interDistLH1);print(interDistLH2);print('endinterDist')

        isVisible1 = True
        isVisible2 = True
        
        # 1st lighthouse
        if(visFactorLH1 > 0):
            for (j, dist) in sortedTriangles[0]:
                if(nmbr_sensor != j):
                    #print("Testing sensor %i vs tris %i: distance of Sensor %f vs triangle %f" % (i
                    #, j, distLH1, dist))
                    tris = triangles[j]
                    newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(tris)), 'lighthouse1')#,j)
                    if(newInterDistLH1 < interDistLH1 and newInterDistLH1 != False):
                        isVisible1 = False
                    if(not isVisible1 or dist > distLH1):
                        # Break if not visible or already checked all tris
                        # that are located closer to the lighthouse that the sensor
                        break
            if(isVisible1):
                visibleLH1 += visFactorLH1
                    
        # 2nd lighthouse
        if(visFactorLH2 > 0):
            for (j, dist) in sortedTriangles[1]:
                if(nmbr_sensor != j):
                    tris = triangles[j]
                    newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(tris)), 'lighthouse2')#,j)
                    if(newInterDistLH2 < interDistLH2 and newInterDistLH2 != False):
                        isVisible2 = False
                    if(not isVisible2 or dist > distLH2):
                        # Break if not visible or already checked all tris
                        # that are located closer to the lighthouse that the sensor
                        break
            if(isVisible2):
                visibleLH2 += visFactorLH2
        #print(newInterDistLH1); print(newInterDistLH2)
    
    fractionVisibleLH1 = float(visibleLH1) / sensorsToDistribute
    fractionVisibleLH2 = float(visibleLH2) / sensorsToDistribute
        
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0.0
    distTemp = 0.0
    distMax = 0.0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                distTemp = np.linalg.norm(ind-indivi)
                dist += distTemp
                if(distTemp > distMax):
                    distMax = distTemp
    #print(dist);print(distMax)
    distNorm = dist/(distMax * sensorsToDistribute)

    #print(visibleLH1);print(visibleLH2);print('')
    
    return (fractionVisibleLH1 + fractionVisibleLH2 + distNorm),

In [ ]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensors.tolist())-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# Creating population

population = toolbox.population(n=100)

In [ ]:
hof = tools.HallOfFame(10)

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [ ]:
from data.algorithmsMod import varAnd
from deap import tools
from data.trafomatrix import getRandomRotationmatrix
from data.bestSensorVis import bestSensorVis

#MODDED VERSION of eaSimple from DEAP
def eaSimpleMod(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm reproduce the simplest evolutionary algorithm as
    presented in chapter 7 of [Back2000]_.
    
    Modded version of DEAP Evolutionary Algorithm Framework
    https://github.com/DEAP/deap
    """
    global sensor_dict
    global triangles
    global orientationMesh
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print logbook.stream

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print logbook.stream
        
        #sensorMovement = tools.selBest(population, k=1)[0]
        #bestSensorVis(sensor_dict, sensorMovement, rate=1000, color=(0,1,0,0.8), sphereSize=0.2)
        
        if(gen%1==0):
            global sensorDictBackup
            global trianglesBackup
            sensor_dict = sensorDictBackup
            R = getRandomRotationmatrix()
            sensorDictNew = []
            
            for sensor in sensor_dict:
                sensorDictNew.append(np.squeeze(np.asarray(R.dot(np.array(sensor[1])))).tolist())
                
            sensor_dict = list(zip(range(len(sensorDictNew)), sensorDictNew))
        
            tri1 = R.dot(np.transpose(trianglesBackup[:,0:3]))
            tri2 = R.dot(np.transpose(trianglesBackup[:,3:6]))
            tri3 = R.dot(np.transpose(trianglesBackup[:,6:9]))

            triangles = np.concatenate((tri1.T,tri2.T,tri3.T),axis=1)
            
            # resort the triangles by distance from lighthouses for speedup
            global sortedTriangles

            lighthouses = [LH1, LH2]
            sortedTriangles = []

            for l in lighthouses:
                tris = trisByMinDistanceSortedMap(triangles, l[0])
                sortedTriangles.append(tris)
                
                

            orientationMesh = Quaternion(matrix=R)
            
            meshVisualization(orientationMesh, stl_file, color=(1.,1.,1.,0.9))
            #sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))
            sensorMovement = tools.selBest(population, k=1)[0]
            bestSensorVis(sensor_dict, sensorMovement, rate=1000, color=(0,1,0,0.8), sphereSize=0.2)
            
            
    return population, logbook

In [ ]:
population, log = eaSimpleMod(population, 
                                toolbox, 
                                cxpb=0.5, 
                                mutpb=0.5, 
                                ngen=150, 
                                stats=stats, 
                                halloffame=hof, 
                                verbose=True)

In [ ]:
bestSensors = tools.selBest(population, k=1)
print(bestSensors[0])

In [ ]:
from data.bestSensorVis import bestSensorVis

#Sensor visualization in RVIZ
orientationMesh = (0,0,0,0)
meshVisualization(orientationMesh, stl_file, color=(1.,1.,1.,0.9))
sensorVisualization(sensorDictBackup, rate=500, sphereSize=0.03, color=(0,0,1,1))
bestSensorVis(sensorDictBackup, bestSensors[0], rate=500, color=(1,0,0,0.8), sphereSize=0.1)